# Clustering sur CAMille

## Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import re
import nltk
import collections
import os
import string
import sys
from pprint import pprint
from scipy.spatial.distance import cosine
from nltk import word_tokenize

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from wordcloud import WordCloud

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ing Armel
[nltk_data]     Fopa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Choix de la décennie et exploration du fichiers CAMille

Verification du dossiers CAMille

In [3]:
data_dir = Path("../../data/txt")

print("Dossier existant?", data_dir.exists())

Dossier existant? True


Recencons les fichiers par decennie


In [ ]:
from collections import defaultdict


data_dir = Path("../../data/txt")
all_txt = sorted(data_dir.glob("*.txt"))

# Dictionnaire décennie : liste de fichiers
decades = defaultdict(list)

pattern = re.compile(r"(\d{4})")

for path in all_txt:
    match = pattern.search(path.name)
    if match:
        year = int(match.group(1))
        decade = (year // 10) * 10
        decades[decade].append(path.name)

# Afficher un résumé clair
print("Nombre de fichiers par décennie :\n")
for dec in sorted(decades.keys()):
    print(f"{dec}s : {len(decades[dec])} fichiers")
    for name in decades[dec]:
        print("   -", name)
    print()


📊 Nombre de fichiers par décennie :

1830s : 1 fichiers
   - KB_JB555_1836-02-08_01-00002.txt

1840s : 1 fichiers
   - KB_JB449_1846-05-30_01-00002.txt

1850s : 3 fichiers
   - KB_JB494_1853-10-30_01-0002.txt
   - KB_JB567_1857-02-02_01-00003.txt
   - KB_JB572_1850-03-15_01-00003.txt

1860s : 2 fichiers
   - KB_JB638_1860-05-21_01-00002.txt
   - KB_JB92_1860-02-09_01-00003.txt

1870s : 1 fichiers
   - KB_JB92_1873-02-06_01-00002.txt

1880s : 4 fichiers
   - KB_JB258_1884-09-03_01-0003.txt
   - KB_JB837_1886-12-28_01-00002.txt
   - KB_JB838_1887-12-28_01-00003.txt
   - KB_JB92_1885-09-29_01-00002.txt

1890s : 7 fichiers
   - KB_JB230_1892-08-07_01-0003.txt
   - KB_JB258_1894-12-09_01-0003.txt
   - KB_JB421_1899-05-15_01-00003.txt
   - KB_JB555_1899-01-19_01-00003.txt
   - KB_JB567_1892-01-03_01-00005.txt
   - KB_JB685_1894-05-14_01-0003.txt
   - KB_JB729_1895-10-08_01-00003.txt

1900s : 4 fichiers
   - KB_JB230_1903-10-16_01-0002.txt
   - KB_JB258_1906-01-09_01-0002.txt
   - KB_JB638_19

Choix de la décennie

In [47]:
DECADE = 1920
files_decade = decades[DECADE]

print(f"Decennie sélectionné : {DECADE}s")
print(f"Nombre de fichiers : {len(files_decade)}")

for name in files_decade:
    print(" -", name)

Decennie sélectionné : 1920s
Nombre de fichiers : 7
 - KB_JB421_1926-10-29_01-00002.txt
 - KB_JB427_1920-01-10_01-00004.txt
 - KB_JB494_1922-09-28_01-0005.txt
 - KB_JB567_1924-08-30_01-00003.txt
 - KB_JB572_1927-07-20_01-00005.txt
 - KB_JB729_1927-11-15_01-00004.txt
 - KB_JB837_1925-01-01_01-00003.txt


Liste et Filtrage des fichiers de la décennie choisie (1950-1959)

In [38]:
# Liste des fichiers .txt
all_txt = sorted(data_dir.glob("*.txt"))

# Filtrage pour la décennie choisie
files_decade = [p for p in all_txt if DECADE in p.name]

print(f"Nombre de fichiers pour la décennie '{DECADE}' : {len(files_decade)} ")
for p in files_decade[:10]:
    print(" -", p.name)

Nombre de fichiers pour la décennie '194' : 6 
 - KB_JB427_1949-07-18_01-00008.txt
 - KB_JB449_1947-08-29_01-00003.txt
 - KB_JB555_1940-03-01_01-00004.txt
 - KB_JB638_1946-07-18_01-00003.txt
 - KB_JB835_1949-09-18_01-00004.txt
 - KB_JB838_1943-09-04_01-00002.txt


In [ ]:
import nltk

nltk.download('punkt')
nltk.download("punkt_tab")

In [ ]:
data_path = "../data/txt/"

## Choisir une décennie

In [ ]:
DECADE = '1950'

## Charger tous les  fichiers de la décennie et en créer une liste de textes

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5]

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos documents

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

## Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

Pour en savoir plus sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

### Définir un nombre de clusters

In [ ]:
N_CLUSTERS = 5

### Instancier le modèle K-Means et ses arguments

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

## Visualiser les clusters

### Réduire les vecteurs à 2 dimensions à l'aide de l'algorithme PCA
Cette étape est nécessaire afin de visualiser les documents dans un espace 2D

https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")